[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/training/simple_dreambooth_trainer_dev.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2
!pip install -q --pre xformers
!pip install -q --pre triton

!pip install diffusers==0.13.1 -U
!pip install transformers==4.26.1 -U
!pip install accelerate==0.16.0 -U
!pip install ftfy==6.1.1 -U
!pip install bitsandbytes==0.37.0 -U
!pip install safetensors==0.2.8 -U

!git clone https://huggingface.co/JosephusCheung/ACertainty /content/model
!mv /content/model/ACertainty.ckpt /content/model/model.ckpt

!mkdir /content/trainer
!mkdir /content/trainer/diffusers
!mkdir /content/trainer/diffusers/dreambooth
!mkdir /content/trainer/diffusers/dreambooth/output_dir
!mkdir /content/trainer/diffusers/dreambooth/class_data_dir

!wget https://raw.githubusercontent.com/camenduru/trainer/main/diffusers/dreambooth/train_dreambooth.py -O /content/trainer/diffusers/dreambooth/train_dreambooth.py
!wget https://raw.githubusercontent.com/camenduru/trainer/main/diffusers/dreambooth/convert_diffusers_to_original_stable_diffusion.py -O /content/trainer/diffusers/dreambooth/convert_diffusers_to_original_stable_diffusion.py

!python -u /content/trainer/diffusers/dreambooth/train_dreambooth.py \
                    --pretrained_model_name_or_path="/content/model"  \
                    --instance_data_dir="/content/drive/MyDrive/AI/training/parkminyoung" \
                    --output_dir="/content/trainer/diffusers/dreambooth/output_dir" \
                    --learning_rate=5e-6 \
                    --max_train_steps=650 \
                    --instance_prompt="parkminyoung" \
                    --resolution=512 \
                    --center_crop \
                    --train_batch_size=1 \
                    --gradient_accumulation_steps=1 \
                    --max_grad_norm=1.0 \
                    --mixed_precision="fp16" \
                    --gradient_checkpointing \
                    --enable_xformers_memory_efficient_attention \
                    --use_8bit_adam \
                    --with_prior_preservation \
                    --class_data_dir="/content/trainer/diffusers/dreambooth/class_data_dir" \
                    --prior_loss_weight=1.0 \
                    --sample_batch_size=2 \
                    --class_prompt="person" \
                    --seed=69 \
                    --num_class_images=12